<a href="https://colab.research.google.com/github/idityaGE/Generative-AI/blob/main/Retrieval%20Augmented%20Generation%20(RAG)/Custom_Website_Chatbot_(simple_RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain langchain-community pypdf sentence_transformers bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install tokenizers faiss-cpu
!pip -q install unstructured
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 63.6 MB/s eta 0:00:00


In [ ]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import notebook_login
from langchain import HuggingFacePipeline

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
URLs = [
    "https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb",
    "https://www.databricks.com/blog/long-context-mpt-7b-8k",
    "https://stability.ai/research/humans-in-4d-reconstructing-and-tracking-humans-with-transformers",
    "https://lmsys.org/blog/2023-10-30-toxicchat/",
]

In [ ]:
loader = UnstructuredURLLoader(urls=URLs)
data = loader.load()

In [ ]:
data[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their app

In [ ]:
# just the play thing
loaderPortfolio = UnstructuredURLLoader(urls=['https://idityage.github.io/'])
portfolioData = loaderPortfolio.load()
portfolioData # Problem here is that its not a static page (I mean when get send the request to url it doesn't send a actuall data within the HTML but rather it run the JS file and then render the HTML page)
# this website is made using React and the HTML file doesn't contains anything, JavaScript run the element after the running the javscript file

[Document(metadata={'source': 'https://idityage.github.io/'}, page_content='')]

In [ ]:
text_splitter = CharacterTextSplitter(separator='\n',chunk_size=1000, chunk_overlap=200)
text_chunk = text_splitter.split_documents(data)

In [ ]:
print(len(text_chunk))
text_chunk[0]

52


Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

In [ ]:
text_chunk[2]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining Data\nThe authors utilized a novel mix of data from publicly accessible sources to train the Llama 2 models, excluding any data from Meta’s products or services. They made efforts to erase data from certain sites known for harboring large amounts of personal information about private individuals. They trained the models on 2 trillion tokens of data, believing this amount provided a beneficial performance-cost balance. They also up-sampled the most factual sources to boost knowledge and reduce instances of false information generation or “hallucinations”.\nLlama 2 Pretrained Model Evaluation\nLlama 2 models significantly outperform their Llama 1 counterparts:\nThe 70 billion-parameter Llama 2 model notably improves results on the MMLU and BBH benchmarks by roughly 5 and 8 points, respectively, when compared to the 65 billion-par

In [ ]:
embeddings = HuggingFaceEmbeddings()
print(len(embeddings.embed_query("Hello World")))

<ipython-input-11-ff3362285d3d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-11-ff3362285d3d>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

768


In [ ]:
import faiss
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

In [ ]:
from langchain_community.docstore.in_memory import InMemoryDocstore
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(data))]
vector_store.add_documents(documents=data, ids=uuids)

['94e2f33a-4ff2-4880-9cb3-428f2f33838e',
 '5bdbd759-2bc8-4977-82d4-51cfc9c3848f',
 '0f9413ac-bb68-45ba-ab3d-b3e129665163',
 'a207b6ae-1093-419b-bda1-831cb83ad4d4']

In [ ]:
notebook_login()

In [ ]:
model = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True,
                                             )

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map='auto',
                max_new_tokens=512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                # Fallback if eos_token_id is not available
                eos_token_id=tokenizer.eos_token_id if hasattr(tokenizer, 'eos_token_id') else None,
                # Add padding token handling
                pad_token_id=tokenizer.pad_token_id if hasattr(tokenizer, 'pad_token_id') else tokenizer.eos_token_id
                )

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

<ipython-input-20-ee300df27e98>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [ ]:
llm.predict("please provide a concise summary of the book Harry Potter")

<ipython-input-21-8de2a7568427>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("please provide a concise summary of the book Harry Potter")


"please provide a concise summary of the book Harry Potter and the Philosopher's Stone by J.K. Rowling.\n\nHarry Potter and the Philosopher's Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter and his friends Ron Weasley and Hermione Granger as they attend Hogwarts School of Witchcraft and Wizardry.\n\nOn Harry's eleventh birthday, he discovers that he is the son of two powerful wizards who were killed by the dark wizard Lord Voldemort. Harry learns that he is famous in the wizarding world for being the only person to survive a killing curse from Voldemort, and that he has been left with a lightning-shaped scar on his forehead as a result.\n\nHarry begins his first year at Hogwarts, where he makes friends with Ron and Hermione and becomes entangled in a mystery surrounding the Philosopher's Stone, a powerful magical object that can grant eternal life. Harry and his friends must navigate the challenges of school and fr

## Intialize the Retrival QA

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
query = "How good is Vicuna ?"

In [ ]:
docs = vector_store.similarity_search(query, k = 3)
print(len(docs))

3


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=vector_store.as_retriever())

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vector_store.as_retriever())

In [ ]:
result=chain({"question": "How Good is Vicuna ?"}, return_only_outputs=True)
result[0]

<ipython-input-27-6d6f6f2660e2>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({"question": "How Good is Vicuna ?"}, return_only_outputs=True)
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (11945 > 1024). Running this sequence through the model will result in indexing errors


ValueError: A single document was longer than the context length, we cannot handle this.

In [ ]:
qa.run("How Good is Vicuna ?")

<ipython-input-28-fe59a1642ef7>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run("How Good is Vicuna ?")
